In [1]:
# for generating the feature vector
import json
from nltk import ngrams
import nltk
from collections import Counter

# for NN using pyTorch
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
def extract_headline_category(category_list):
    headline_category = []
    try:
        input_file = open('./News_Category_Dataset.json')
        input_data = input_file.readlines()
        input_file.close()
        for json_object in input_data:
            data = json.loads(json_object)
            category = data['category'].upper()
            if category in category_list:
                headline_category.append((data['headline'], data['category']))
        return headline_category
    except IOError:
        print("ERROR : IO ERROR occurred while opening file")
        exit(0)

In [3]:
category_list = ['Business', 'Comedy','Sports', 'Crime', 'Religion']
category_list = [i.upper() for i in category_list]
headlines_and_category = extract_headline_category(category_list)

In [4]:
def get_n_grams(dataset:'headline, category', n:'n gram value', k:'return top k n-grams'):
    n_grams_list=[]
    for headline, category in dataset:
        tokenize = nltk.word_tokenize(headline)
        n_gram = nltk.ngrams(tokenize, n)
        n_grams_list.extend(n_gram)
    print(len(n_grams_list))
    top_k = Counter(n_grams_list).most_common(k)
    most_frequent=[i[0] for i in top_k]
    return most_frequent

In [5]:
unigrams_dict = get_n_grams(headlines_and_category, 1, 500)
bigrams_dict = get_n_grams(headlines_and_category, 2, 300)
trigrams_dict = get_n_grams(headlines_and_category, 3, 200)

187445


/Users/abhishekagrawal/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: generator 'ngrams' raised StopIteration
  


169606
151792


In [6]:
pos_list = list({'CD', 'CC', 'RP', 'NNPS', 'IN', ',', '$', 'FW', 'RBR', 'JJ', "''", ')', 'VBD', 'VBP', 'POS', ':', 'NNS', '#', 'PRP', '(', 'VBN', 'PDT', 'JJS', 'VBG', 'PRP$', 'RBS', 'LS', '.', 'EX', 'NN', '``', 'DT', 'RB', 'WDT', 'VB', 'UH', 'TO', 'JJR', 'VBZ', 'MD', 'NNP', 'WP', 'WRB'})

In [7]:
def generate_features(dataset, unigrams_dict, bigrams_dict, trigrams_dict, pos_list, category_list):
    X = []
    y = []
    for headline, category in dataset:
        text = nltk.word_tokenize(headline)
        unigrams = nltk.ngrams(text, 1)
        bigrams = nltk.ngrams(text, 2)
        trigrams = nltk.ngrams(text, 3)
#         list_of_ngrams_dict = [unigrams_dict]
#         list_of_ngrams_sentences = [unigrams]
        list_of_ngrams_dict = [unigrams_dict, bigrams_dict, trigrams_dict]
        list_of_ngrams_sentences = [unigrams, bigrams, trigrams]
        temp_sentence = []
        for i, ngram in enumerate(list_of_ngrams_dict):
            temp_ngram = [0]*len(ngram)
            for word in list_of_ngrams_sentences[i]:
                if word in ngram:
                    temp_ngram[ngram.index(word)]+=1
            temp_sentence.extend(temp_ngram)
        
    
#         temp_pos = [0]*len(pos_list)
#         for word, tag in nltk.pos_tag(text):
#             temp_pos[pos_list.index(tag)]+=1
#         temp_sentence.extend(temp_pos)
        X.append(temp_sentence)
        y.append(category_list.index(category))
    return (X,y)

In [8]:
X, y = generate_features(headlines_and_category, unigrams_dict, bigrams_dict, trigrams_dict, pos_list, category_list)

/Users/abhishekagrawal/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: DeprecationWarning: generator 'ngrams' raised StopIteration
  app.launch_new_instance()


In [9]:
y_temp = []
for i in y:
    temp = [0, 0, 0, 0, 0]
    temp[i] = 1
    y_temp.append(temp)
y = y_temp

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

inputs = list(map(lambda s: Variable(torch.Tensor([s])), X_train))
targets = list(map(lambda s: Variable(torch.Tensor([s])), y_train))
in_features = len(X_train[0])
out_features = len(category_list)

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.main = nn.Sequential(
            nn.Linear(in_features, 64),
            nn.Sigmoid(),
            nn.Linear(64, out_features),
        )

    def forward(self, input):
        return self.main(input)

In [ ]:
net = Net()
criterion = nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr=0.01)

EPOCHS_TO_TRAIN = 50
best_model = net
best_loss = 1
print("Training loop:")
for idx in range(0, EPOCHS_TO_TRAIN):
    for input, target in zip(inputs, targets):
        optimizer.zero_grad()   # zero the gradient buffers
        output = net(input)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()    # Does the update
    if loss.data.numpy() < best_loss:
        best_model = net
    print("Epoch {: >8} Loss: {}".format(idx, loss.data.numpy()))

Training loop:
Epoch        0 Loss: 0.20865678787231445
Epoch        1 Loss: 0.19925135374069214
Epoch        2 Loss: 0.20160391926765442
Epoch        3 Loss: 0.2047022134065628
Epoch        4 Loss: 0.20484842360019684
Epoch        5 Loss: 0.203580841422081
Epoch        6 Loss: 0.20252321660518646
Epoch        7 Loss: 0.20192097127437592
Epoch        8 Loss: 0.20166201889514923
Epoch        9 Loss: 0.2016243189573288
Epoch       10 Loss: 0.20171670615673065
Epoch       11 Loss: 0.20187704265117645
Epoch       12 Loss: 0.20206648111343384
Epoch       13 Loss: 0.20226314663887024
Epoch       14 Loss: 0.2024555504322052
Epoch       15 Loss: 0.20263853669166565
Epoch       16 Loss: 0.20281003415584564
Epoch       17 Loss: 0.20296931266784668
Epoch       18 Loss: 0.2031165212392807
Epoch       19 Loss: 0.20325231552124023
Epoch       20 Loss: 0.20337705314159393


In [ ]:
inputs = list(map(lambda s: Variable(torch.Tensor([s])), X_test))
targets = list(map(lambda s: Variable(torch.Tensor([s])), y_test))

In [ ]:
print("")
print("Final results:")
total_accurate = 0
for inp, target in zip(inputs, targets):
    output = best_model(inp)
    temp = target[0].detach().numpy().tolist()
    temp_2 = output[0].detach().numpy().tolist()
    if temp_2.index(max(temp_2)) == temp.index(max(temp)):
        total_accurate += 1
print(total_accurate / len(inputs))